In [421]:
#compute the eigenvectors of Schur's matrix according to Morton's '78 paper
#"On the Eigenvectors of Schur's Matrix": https://www.sciencedirect.com/science/article/pii/0022314X80900839
#NOTE: The DFT matrix is subtly different than the Schur matrix. DFT = exp(2*pi*i*m*n/q) for 0 <= m,n <= q-1
#whereas for Schur's matrix it is 1 <= m,n <= q

In [513]:
#compute the eigenvectors of the unitary matrix corresponding to the Fourier transform over the complex numbers
q=3
alpha = exp(2*pi*I/q)
#K.<alpha> = CyclotomicField(3)
U = matrix([[alpha**(i*j) for j in range(1,q+1)] for i in range(1,q+1)])
U.eigenvectors_right()

[(I*sqrt(3), [(1, -1, 0)], 1),
 (-sqrt(3), [(1, 1, -sqrt(3) + 1)], 1),
 (sqrt(3), [(1, 1, sqrt(3) + 1)], 1)]

In [542]:
#define the vectors X_d given a divisor d of q/f(chi), where f(chi) is the conductor of the character \chi
def X(d,chi):
    q = chi.modulus()
    assert d.divides(q/chi.conductor())
    return vector([chi.primitive_character()(n/d) if d.divides(n) else 0 for n in range(1,q+1)])

In [498]:
#compute eigenvectors given a character chi and a divisor d of q/f(chi)
def eigenvector(d,chi):
    try:
        assert d.divides(q/chi.conductor())
    except AssertionError:
        print("Error: d must divide q/f(chi)")
        return

    #define the eigenvectors E(chi,d,lambda) as in (13) in the paper
    conjugate_divisor = Integer(chi.modulus()/(chi.conductor()*d))
    eig_plus = sqrt(chi(-1)*chi.modulus())
    eig_minus = -sqrt(chi(-1)*chi.modulus())
    #need to restrict character to modulus conductor, i.e. the primitive character
    E = lambda chi, d, eig: sqrt(d)*X(d,chi) + (eig/(chi.bar().primitive_character().gauss_sum()*sqrt(d)))*conjugate(X(conjugate_divisor,chi))

    #check if chi is real and if the ratio of modulus to conductor is the square of a divisor
    reality_condition = chi.bar() == chi
    square_condition = d^2 == chi.modulus()/chi.conductor()

    #if chi is real and square condition is met, just return X_d(chi)
    if reality_condition and square_condition:
        return X(d,chi)

    #if one or the other condition is met, return a pair of eigenvalues corresponding to E(chi,d,\pm lambda)
    if reality_condition != square_condition:
        return E(chi,d,eig_plus), E(chi,d,eig_minus)

    #if neither condition are met, return four eigenvalues E(chi,d,\pm lambda) and E(chi_bar,d,\pm lambda)
    if (not reality_condition) and (not square_condition):
        return E(chi,d,eig_plus), E(chi,d,eig_minus), E(chi.bar(),d,eig_plus), E(chi.bar(),d,eig_minus)

In [593]:
from sage.misc.flatten import flatten
#compute all the eigenvectors the Schur matrix
def all_eigenvectors(q):
    all_eigs = []
    G = DirichletGroup(q)
    for chi in G:
        f = chi.conductor()
        for d in divisors(q/f):
            all_eigs.append(eigenvector(Integer(d),chi))
    return flatten(all_eigs)

In [596]:
all_eigenvectors(3)

[(1, 1, sqrt(3) + 1),
 (1, 1, -sqrt(3) + 1),
 (1, 1, sqrt(3) + 1),
 (-1, -1, sqrt(3) - 1),
 (1, -1, 0)]